In [ ]:
!apt install libomp-dev
!pip install faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (417 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Se

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import numpy as np
import sys
sys.path.append('./function')
sys.path.append('./function/skmultilearn')
import csv
from sklearn.preprocessing import MinMaxScaler
from skmultilearn.adapt import MLkNN

In [ ]:
import numpy as np
from liblinear.python.liblinear.liblinearutil import *
from PML_NI import PML_NI
def functions(train_data,train_target,true_target,test_data,test_target,mode):
  if mode == 0: ## PML_NI
    clf = PML_NI.PMLNI()
    clf.fit(train_data,train_target,true_target,test_data)
    creds = clf.outputs()
    predict_labels = clf.predict()
  elif mode == 1: ## ML_kNN
    """model = mlk.multilabel_kNN(k=3, metric = "cosine") #k: number of neighbors, metric: distance metric {"euclidean", "cosine"}
    model.fit(train_data, train_target) # X :2d feature vectors. Y: label matrix, where Y[i,k] = 1 if i has label k.
    predict_labels = model.predict(test_data).toarray() # X :2d feature vectors. Y: label matrix, where Y[i,k] = 1 if i has label k.
    creds = model.predict(test_data, return_prob = True)[1].toarray()"""
    clf = MLkNN(k=3)
    clf.fit(train_data, train_target)
    predict_labels = clf.predict(test_data).toarray()
    creds = clf.predict_proba(test_data).toarray()
  elif mode == 2:
    train_target_T = np.array(train_target).T
    test_target_T = np.array(test_target).T
    [num_label,num_train] = np.shape(train_target_T)
    [num_test,num_fea] = np.shape(test_data)
    #%cd /content/drive/My Drive/Colab Notebooks/function/liblinear-2.45
    creds = np.zeros((num_label,num_test))
    predict_labels = np.zeros((num_label,num_test))
    for i in range(0,num_label):
      y_train = train_target_T[i]
      y_test = test_target_T[i]
      m = train(y_train, train_data)
      p_label, p_acc, p_val = predict(y_test, test_data, m)
      predict_labels[i] = np.array(p_label)
      creds[i] = np.ravel(np.array(p_val))
  return(creds,predict_labels)

def get_cvdata(data,candidate_labels,target,crossval_ind,cv_num):
  train_data = data[np.where(crossval_ind != cv_num)]
  train_target = candidate_labels[np.where(crossval_ind != cv_num)]
  true_target = target[np.where(crossval_ind != cv_num)]
  test_data = data[np.where(crossval_ind == cv_num)]
  test_target = target[np.where(crossval_ind == cv_num)]
  return(train_data,train_target,true_target,test_data,test_target)

def save_result(kind_of_data,creds,predict_labels,mode,percent_of_noise,p_true,cv_num):
  func_list = ["PML_NI","ML_kNN","BR"]
  dir_name = "./result/"+str(func_list[mode])+"/"+str(kind_of_data)+"/"+str(percent_of_noise)+"/true/"+str(p_true)
  np.savetxt(dir_name+"/creds_"+str(cv_num)+".csv",creds,delimiter=",")
  np.savetxt(dir_name+"/predict_labels_"+str(cv_num)+".csv",predict_labels,fmt="%d",delimiter=",")

In [ ]:
#data_list = ["emotions","enron","CAL500","scene","genbase","mirflickr","music_style","music_emotion"]
data_list = ["genbase","mirflickr","music_style","music_emotion"]
data_list = ["music_emotion"]
true_list=[1,3,5,7,9]
#true_list=[1]
noise_list = [50,100,150]
#noise_list = [50]
#func_list = ["ML_kNN","BR"]
#func_list = ["PML_NI"]

In [ ]:
for kind_of_data in data_list:
  print(kind_of_data)
  percent_of_noise = 0
  data_path = f"/content/drive/MyDrive/Colab Notebooks/new_data2/" + kind_of_data + "/"
  data = np.loadtxt(data_path+"data.csv", delimiter=',')
  #data = np.loadtxt("./datas/"+str(kind_of_data)+"/0/"+data_fname, delimiter=',')
  target = np.loadtxt(data_path+"target.csv", delimiter=',',dtype = float).astype(int)
  crossval_ind = np.loadtxt(data_path+"index/5-cv.csv",delimiter=',',dtype = int)-1
  for mode in range(2,3):

    for cv_num in range(1,5): # cross validation

      if kind_of_data in ["emotions","enron","CAL500","scene","genbase"]:
        for p_noise in noise_list: # percent_of_noise
          for p_true in true_list:
            candidate_labels = np.loadtxt(data_path+"cand/"+str(p_noise)+".csv", delimiter=',',dtype = float).astype(int)
            true_inds = np.loadtxt(data_path+"index/true/"+str(p_true)+"/A.csv",delimiter=',',dtype=int)-1
            for t in range(len(true_inds)):
              candidate_labels[true_inds[t]]=target[true_inds[t]]
            print(f"{kind_of_data}/{mode}/cv:{str(cv_num)}/noise:{str(p_noise)}/true:{str(p_true)}/")
            train_data,train_target,true_target,test_data,test_target = get_cvdata(data,candidate_labels,target,crossval_ind,cv_num)
            creds,predict_labels = functions(train_data,train_target,true_target,test_data,test_target,mode)
            save_result(kind_of_data,creds,predict_labels,mode,p_noise,p_true,cv_num)
      
      elif kind_of_data in ["mirflickr","music_style","music_emotion","YeastBP"]:
        p_noise = str(0)
        for p_true in true_list:
          candidate_labels = np.loadtxt(data_path+"cand/"+str(p_noise)+".csv", delimiter=',',dtype = float).astype(int)
          true_inds = np.loadtxt(data_path+"index/true/"+str(p_true)+"/A.csv",delimiter=',',dtype=int)-1
          for t in range(len(true_inds)):
            candidate_labels[true_inds[t]]=target[true_inds[t]]
          print(f"{kind_of_data}/{mode}/cv:{str(cv_num)}/noise:{str(p_noise)}/true:{str(p_true)}/")
          train_data,train_target,true_target,test_data,test_target = get_cvdata(data,candidate_labels,target,crossval_ind,cv_num)
          creds,predict_labels = functions(train_data,train_target,true_target,test_data,test_target,mode)
          save_result(kind_of_data,creds,predict_labels,mode,p_noise,p_true,cv_num)

music_emotion
music_emotion/2/cv:1/noise:0/true:1/
Accuracy = 78.8588% (1078/1367) (classification)
Accuracy = 87.6372% (1198/1367) (classification)
Accuracy = 47.3299% (647/1367) (classification)
Accuracy = 60.9364% (833/1367) (classification)
Accuracy = 39.7952% (544/1367) (classification)
Accuracy = 65.6181% (897/1367) (classification)
Accuracy = 39.5757% (541/1367) (classification)
Accuracy = 62.8383% (859/1367) (classification)
Accuracy = 86.1741% (1178/1367) (classification)
Accuracy = 76.4448% (1045/1367) (classification)
Accuracy = 92.1726% (1260/1367) (classification)
music_emotion/2/cv:1/noise:0/true:3/
Accuracy = 78.8588% (1078/1367) (classification)
Accuracy = 87.7103% (1199/1367) (classification)
Accuracy = 48.793% (667/1367) (classification)
Accuracy = 62.18% (850/1367) (classification)
Accuracy = 39.9415% (546/1367) (classification)
Accuracy = 67.8859% (928/1367) (classification)
Accuracy = 43.9649% (601/1367) (classification)
Accuracy = 63.7893% (872/1367) (classificati

In [ ]:
a=np.array([1.2,1.3])
b=a.astype(int)
print(b)

[1 1]


In [ ]:
for kind_of_data in data_list:
  for func in func_list:
    if kind_of_data in ["emotions","enron","CAL500","scene","genbase"]:
      for p_noise in noise_list:
        %cd /content/drive/MyDrive/Colab Notebooks/result/{func}/{kind_of_data}/{str(p_noise)}
        #%mkdir true
        %cd /content/drive/MyDrive/Colab Notebooks/result/{func}/{kind_of_data}/{str(p_noise)}/true
        for p_true in true_list:
          %mkdir {str(p_true)}
    """else:
      p_noise=0
      %cd /content/drive/MyDrive/Colab Notebooks/result/{func}/{kind_of_data}/{str(p_noise)}
      #%mkdir true
      %cd /content/drive/MyDrive/Colab Notebooks/result/{func}/{kind_of_data}/{str(p_noise)}/true
      for p_true in true_list:
        %mkdir {str(p_true)}"""